In [4]:
import numpy as np
import random
import torch
import torch.utils.data as data
from PIL import Image
from PIL import Image, ImageFont, ImageDraw, ImageEnhance

import os
import os.path
import colorsys

from pycocotools.coco import COCO
from pycocotools import mask as maskUtils

inf = float('inf')
nan = float('nan')

In [5]:
train_dir = "/home/aravind/dataset/train2017"
val_dir = "/home/aravind/dataset/val2017"
train_ann = "/home/aravind/dataset/annotations/instances_train2017.json"
val_ann = "/home/aravind/dataset/annotations/instances_val2017.json"

In [13]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 16
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 16
    PIN_MEMORY = True
    VALIDATION_STEPS = 20
    # including bg
    NUM_CLASSES = 81

    MEAN_PIXEL = np.array(
        [0.485, 0.456, 0.406], dtype=np.float32).reshape(1, 1, -1)
    STD_PIXEL = np.array(
        [0.229, 0.224, 0.225], dtype=np.float32).reshape(1, 1, -1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    GRID_SHAPE = 11
    IMPULSE_SHAPE = (48, 48)
    MIN_PIXELS = 256
    MIN_INTERSECTION = 128
    def __init__(self):
        self.WIDTH = 32 * self.GRID_SHAPE
        self.HEIGHT = 32 * self.GRID_SHAPE
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT, 3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [17]:
class CocoDetection(data.Dataset):
    def __init__(self, root, annFile, config):
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(self.coco.imgs.keys())
        self.config = config
        self.catMap = self.build_class_map()

    # coco ids remapped to contigous range(81) (including background as 0)
    def build_class_map(self):
        catMap = {}
        coco_cat_ids = [0] + self.coco.getCatIds(config.CLASS_NAMES[1:])
        for i in range(81):
            catMap[coco_cat_ids[i]] = i
        return catMap

    def __getitem__(self, index):
        # IO stuff: reading image, masks; decoding masks as numpy arrays
        img, instance_masks, class_ids = self.load_data(index)

        # Data Augmentation:
        # skip for now

        # Target generation:
        return self.generate_targets(img, instance_masks, class_ids)

    # make base impulse
    # map each impulse to object with highest overlap

    def generate_targets(self, img, instance_masks, class_ids):
        # add background mask, background class
        # so that all impulses get some response
        bg_mask = np.where(np.sum(instance_masks, 0, keepdims=True) == 0, 1, 0)
        instance_masks = np.concatenate([bg_mask, instance_masks], 0)
        class_ids = np.concatenate([[0], class_ids], 0)
        # resize image, masks to 448*448
        w, h = self.config.WIDTH, self.config.HEIGHT
        img = self.resize_image(img, (w, h), "RGB")
        instance_masks = np.array(
            [self.resize_image(m, (w, h), "L") for m in instance_masks])
        # generate base impulse
        base_impulse = self.make_base_impulse()
        # map masks, class labels to impulses
        # map_freq is number of a single mask is mapped to some impulse
        # this is to normalize loss function
        mask_response, class_response, freq_normalization = self.map_impulse_response(
            base_impulse, instance_masks, class_ids)
        return img, mask_response, class_response, base_impulse, freq_normalization

    # not so fast implementation of iou between two pairs of masks
    # a, b binary masks [n,w,h]

    def all_pairs_iou(self, a, b):
        iou = np.zeros((a.shape[0], b.shape[0]))
        for i in range(a.shape[0]):
            m = np.expand_dims(a[i], 0)
            intersection = np.sum(m * b, (1, 2))
            intersection = (intersection >
                            self.config.MIN_INTERSECTION) * intersection
            union = np.sum(b + m, (1, 2)) - intersection
            iou[i, :] = intersection / union
        return iou

    def map_impulse_response(self, base_impulse, instance_masks, class_ids):
        scores = self.all_pairs_iou(base_impulse, instance_masks)
        ids = np.argmax(scores, -1)
        mask_response = instance_masks[ids]
        class_response = class_ids[ids]

        freq_normalization = np.ones(base_impulse.shape[0])
        counts = np.zeros(instance_masks.shape[0])
        for i in range(ids.shape[0]):
            counts[ids[i]] += 1
        for i in range(ids.shape[0]):
            freq_normalization[i] = 1 / counts[ids[i]]

        return mask_response, class_response, freq_normalization

    # we generate impulses evenly spread across the image
    # divide image into grid; this is heuristic
    # here, we place impulse centres as from d to (2g-1)*d
    # separated by 2*d. where d = w//(2*g)

    def make_base_impulse(self):
        config = self.config
        g = config.GRID_SHAPE
        w, h = config.WIDTH, config.HEIGHT
        dw, dh = w // (2 * g), h // (2 * g)

        base_impulse = np.zeros((g * g, w, h))
        dx, dy = config.IMPULSE_SHAPE[0] // 2, config.IMPULSE_SHAPE[1] // 2

        for i in range(g):
            for j in range(g):
                k = g * i + j
                x, y = dw * (2 * i + 1), dh * (2 * j + 1)
                lx = max(0, x - dx)
                ly = max(0, y - dy)
                rx = min(x + dx, w)
                ry = min(y + dy, h)
                base_impulse[k][lx:rx,ly:ry] = np.ones((rx-lx,ry-ly))
        return base_impulse

    # resize image/mask to specified size without losing aspect ratio

    def resize_image(self, img, size, mode):
        interpolation = {"RGB": Image.BICUBIC, "L": Image.NEAREST}[mode]
        img_obj = Image.fromarray(img.astype(np.uint8), mode)
        img_obj.thumbnail(size, interpolation)

        (w, h) = img_obj.size
        padded_img = Image.new(mode, size, "black")
        padded_img.paste(img_obj, ((size[0] - w) // 2, (size[1] - h) // 2))

        return np.array(padded_img)

    # read image, masks; decode masks to numpy arrays
    # image format: channels last
    # mask format: channels first

    def load_data(self, index):
        coco = self.coco
        config = self.config

        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(
            imgIds=img_id, areaRng=[config.MIN_PIXELS, inf], iscrowd=False)
        anns = coco.loadAnns(ann_ids)
        path = coco.loadImgs(img_id)[0]['file_name']

        img = Image.open(os.path.join(self.root, path)).convert('RGB')
        instance_masks = []
        class_ids = []
        w, h = img.size
        for ann in anns:
            instance_masks.append(self.annToMask(ann, h, w))
            class_ids.append(self.catMap[ann['category_id']])
        return np.array(img), np.array(instance_masks), np.array(class_ids)

    def __len__(self):
        return len(self.ids)

    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        fmt_str += '    Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(
            tmp,
            self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(
            tmp,
            self.target_transform.__repr__().replace('\n',
                                                     '\n' + ' ' * len(tmp)))
        return fmt_str

    def annToRLE(self, ann, h, w):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """

        segm = ann['segmentation']
        if type(segm) == list:
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, h, w)
            rle = maskUtils.merge(rles)
        elif type(segm['counts']) == list:
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, h, w)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, h, w):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann, h, w)
        m = maskUtils.decode(rle)
        return m

In [18]:
config = ProposalConfig()
val_dataset = CocoDetection(val_dir, val_ann, config)
# train_dataset = CocoDetection(train_dir,train_ann)

loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
